In [ ]:
# 参考 https://github.com/AnthonyK97/Text-Classification-on-IMDB
# https://github.com/AnthonyK97/Text-Classification-on-IMDB/blob/main/2%20CNN%2BGlove.ipynb

import os
import sys
import random
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import OrderedDict
import re, string
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

def seed_everything(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # some cudnn methods can be random even after fixing the seed
    # unless you tell it to be deterministic
    torch.backends.cudnn.deterministic = True
    
!mkdir ./model_bakup/

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class CFG:
    batch_size = 20
    lr = 0.02
    eval_step_num = 50
    mid_eval = False
    best_eval_acc = 0.0
    model_output_dir = './model_bakup/'
    seed = 2033
    use_ema = False
    use_adversial_training = False
    
DEBUG_RUN = True

global_start_t = time.time()
print('ok')

In [ ]:
seed_everything(seed=42)

imdb_data = pd.read_csv('../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
imdb_data['sentiment'] = imdb_data['sentiment'].map({'positive': 1, 'negative': 0})
print('before drop_duplicates, imdb_data.shape: ', imdb_data.shape)
imdb_data = imdb_data.drop_duplicates()
print('after drop_duplicates, imdb_data.shape: ', imdb_data.shape)
imdb_data = imdb_data.sample(30000)
print('after sample, imdb_data.shape: ', imdb_data.shape)
imdb_data = imdb_data.sample(len(imdb_data)).reset_index(drop=True)  # shuffle

imdb_data.head(5)

In [ ]:
MAX_WORDS = 10000   # 仅考虑最高频的10000个词
MAX_LEN = 200
word_count_dict = {}

def clean_text(text):
    lowercase = text.lower().replace('\n', ' ')
    stripped_html = re.sub('<br />', ' ', lowercase)
    cleaned_punctuation = re.sub('[%s]'%re.escape(string.punctuation), '', stripped_html)
    return cleaned_punctuation

for review in imdb_data['review'].values:
    cleaned_text = clean_text(review)
    for word in cleaned_text.split(' '):
        word_count_dict[word] = word_count_dict.get(word, 0) + 1
            
df_word_dict = pd.DataFrame(pd.Series(word_count_dict, name='count'))
df_word_dict = df_word_dict.sort_values(by='count', ascending=False)

df_word_dict = df_word_dict[:MAX_WORDS-2]     # 总共取前max_words-2个词
df_word_dict['word_id'] = range(2, MAX_WORDS)

word_id_dict = df_word_dict['word_id'].to_dict()
word_id_dict['<unknown>'] = 0
word_id_dict['<padding>'] = 1

df_word_dict.head(15)

In [ ]:
def pad(data_list, pad_length):
    padded_list = data_list.copy()
    
    if len(data_list) > pad_length:
        padded_list = data_list[-pad_length:]
        
    if len(data_list) < pad_length:
        padded_list = [1] * (pad_length-len(data_list)) + data_list
        
    return padded_list

def text_to_token(text):
    cleaned_text = clean_text(text)
    word_token_list = [word_id_dict.get(word, 0) for word in cleaned_text.split(' ')]
    pad_list = pad(word_token_list, MAX_LEN)
    token = ' '.join([str(x) for x in pad_list])
    return token
            
process_start_t = time.time()
print('start processing...')
imdb_data['review_tokens'] = imdb_data['review'].map(text_to_token)
print('ok, cost time: ', time.time()-process_start_t)
imdb_data.head(5)

In [ ]:
print(imdb_data['review'].values[0])

In [ ]:
TRAIN_NUM = 15000
imdb_data_test = imdb_data.iloc[:5000]
imdb_data_valid = imdb_data.iloc[5000:10000]
imdb_data_train = imdb_data.iloc[10000:TRAIN_NUM+10000]

if DEBUG_RUN:
    SAMPLE_NUM = 3000
    imdb_data_test = imdb_data_test.sample(SAMPLE_NUM)
    imdb_data_valid = imdb_data_valid.sample(SAMPLE_NUM)
    imdb_data_train = imdb_data_train.sample(2*SAMPLE_NUM)

print(f'imdb_data_train.shape: {imdb_data_train.shape}, imdb_data_valid.shape: {imdb_data_valid.shape}, '
      f'imdb_data_test.shape: {imdb_data_test.shape}')

In [ ]:
cfg = CFG()
seed_everything(seed=cfg.seed)

print('ok')

In [ ]:
class imdbDataset(Dataset):
    def __init__(self, data_df):
        self.data_df = data_df
        
    def __len__(self):
        return len(self.data_df)
    
    def __getitem__(self, index):
        label = self.data_df.iloc[index]['sentiment']
        label = torch.tensor([float(label)], dtype=torch.float, device=device)
        
        tokens = self.data_df.iloc[index]['review_tokens']
        feature = torch.tensor([int(x) for x in tokens.split(' ')], dtype=torch.long, device=device)
            
        return feature, label
    
def generate_data_iter(cfg):
    global imdb_data_train, imdb_data_valid, imdb_data_test
    ds_train = imdbDataset(imdb_data_train)
    ds_valid = imdbDataset(imdb_data_valid)
    ds_test = imdbDataset(imdb_data_test)
    print('len of ds_train: ', len(ds_train), 'len of ds_valid: ', len(ds_valid),
          'len of ds_test: ', len(ds_test))

    dl_train = DataLoader(ds_train, batch_size=cfg.batch_size, shuffle=True, num_workers=0)
    dl_valid = DataLoader(ds_valid, batch_size=cfg.batch_size, shuffle=False, num_workers=0)
    dl_test = DataLoader(ds_test, batch_size=cfg.batch_size, shuffle=False, num_workers=0)
    return dl_train, dl_valid, dl_test

dl_train, dl_valid, dl_test = generate_data_iter(cfg)
print('ok')

In [ ]:
class model_EMA:
    '''
    # https://zhuanlan.zhihu.com/p/68748778
    Example
    # 初始化
    ema = EMA(model, 0.999)

    # 训练阶段，更新完参数后

    '''
    def __init__(self, model, decay=0.99):
        self.model = model
        self.decay = decay
        self.registered = False
        self.shadow = {}
        self.backup = {}

    def is_registered(self):
        return self.registered

    def register(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                self.shadow[name] = param.data.clone()
        self.registered = True

    def update(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                new_average = (1.0 - self.decay) * param.data + self.decay * self.shadow[name]
                self.shadow[name] = new_average.clone()

    def apply_shadow(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                self.backup[name] = param.data
                param.data = self.shadow[name]

    def restore(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.backup
                param.data = self.backup[name]
        self.backup = {}
        
class FGM():
    '''
    Example
    # 初始化
    fgm = FGM(model,epsilon=1,emb_name='word_embeddings.')
    for batch_input, batch_label in data:
        # 正常训练
        loss = model(batch_input, batch_label)
        loss.backward() # 反向传播，得到正常的grad
        # 对抗训练
        fgm.attack() # 在embedding上添加对抗扰动
        #model.zero_grad()  # 如果需要两次回传梯度不累加, 只使用后面添加扰动之后的得到的梯度，则去掉该行的注释！
        loss_adv = model(batch_input, batch_label)
        loss_adv.backward() # 反向传播，并在正常的grad基础上，累加对抗训练的梯度
        fgm.restore() # 恢复embedding参数
        # 梯度下降，更新参数
        optimizer.step()
        model.zero_grad()
    '''
    def __init__(self, model, emb_name, epsilon=1.0, adv_random=False):
        # emb_name这个参数要换成你模型中embedding的参数名
        self.model = model
        self.epsilon = epsilon
        self.emb_name = emb_name
        self.adv_random = adv_random
        self.backup = {}

    def attack(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad and self.emb_name in name:
                #print('found an param: ', name)
                self.backup[name] = param.data.clone()
                norm = torch.norm(param.grad)
                #print('in attack() norm is ', norm)
                #print('param.data: ', param.data, 'param.grad: ', param.grad)
                #print('in attack() norm.shape is ', norm.shape, 'param.data.shape: ', param.data.shape, 'param.grad.shape: ', param.grad.shape)
                if norm!=0 and not torch.isnan(norm):
                    epsilon = self.epsilon
                    if self.adv_random:
                        epsilon *= random.uniform(0.5, 1.5)
                    r_at = epsilon * param.grad / norm
                    #r_at = 0.1 * random.uniform(0.5, 1.5) * param.grad
                    #r_at = 0.1 * param.grad
                    param.data.add_(r_at)

    def restore(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad and self.emb_name in name:
                assert name in self.backup
                param.data = self.backup[name]
        self.backup = {}
        
print('ok')

In [ ]:
EMBEDDING_DIM = 100

class CNN_Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.embedding = nn.Embedding(num_embeddings=MAX_WORDS, embedding_dim=EMBEDDING_DIM, padding_idx=1)
        
        self.conv = nn.Sequential()
        self.conv.add_module('conv_1', nn.Conv1d(in_channels=EMBEDDING_DIM, out_channels=16, kernel_size=5))
        self.conv.add_module('pool_1', nn.MaxPool1d(kernel_size=2))
        self.conv.add_module('relu_1', nn.ReLU())
        self.conv.add_module('conv_2', nn.Conv1d(in_channels=16, out_channels=128, kernel_size=2))
        self.conv.add_module('pool_2', nn.MaxPool1d(kernel_size=2))
        self.conv.add_module('relu_2', nn.ReLU())
        
        self.dense = nn.Sequential()
        self.dense.add_module('flatten', nn.Flatten())
        self.dense.add_module('linear', nn.Linear(6144, 1))
        self.dense.add_module('sigmoid', nn.Sigmoid())
        
    def forward(self, x):
        x = self.embedding(x).transpose(1, 2)
        x = self.conv(x)
        y = self.dense(x)
        return y
    
model = CNN_Net()
print(model)
model.to(device)     

model_param_num = sum(p.numel() for p in model.parameters())
model_trainable_param_num = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('model_param_num: ', model_param_num, 'model_trainable_param_num: ', 
      model_trainable_param_num)

print('ok')

In [ ]:
def accuracy(y_pred, y_true):
    if type(y_pred)==list:
        y_pred = np.array(y_pred)
    y_pred = (y_pred > 0.5)
    if type(y_true)==list:
        y_true = np.array(y_true)
    acc = (y_pred==y_true).mean()
    return acc

def evaluate(model, dl_test, device):
    global cfg
    model.eval()
    
    y_true_lst, y_pred_lst = [], []
    with torch.no_grad():
        for step, batch in enumerate(dl_test):
            feature, label = batch
            feature, label = feature.to(device), label.to(device)
            y_pred = model(feature)
            y_pred_lst += list(y_pred.detach().cpu().numpy())
            y_true_lst += list(label.detach().cpu().numpy())
            
    model.train() # 恢复模型为训练状态
    acc = accuracy(y_pred_lst, y_true_lst)

    return acc
    
def train(model, dl_train, optimizer, loss_func, device):
    global cfg, global_step_num, global_best_valid_acc, dl_valid,  model_ema, fgm
    model.train()  # 将模型置为训练状态
    
    y_true_lst, y_pred_lst = [], []
    for step, batch in enumerate(dl_train):
        global_step_num += 1
        feature, label = batch
        feature, label = feature.to(device), label.to(device)
        y_pred = model(feature)
        train_loss = loss_func(y_pred, label)
        y_pred_lst += list(y_pred.detach().cpu().numpy())
        y_true_lst += list(label.detach().cpu().numpy())
        train_loss.backward()
        optimizer.step()
        model.zero_grad()
        
        if cfg.mid_eval and (global_step_num % cfg.eval_step_num == 0):
            valid_acc = evaluate(model, dl_valid, device)
            print(f'step_num: {global_step_num}, valid_acc: {valid_acc:.5f}')
            if valid_acc > global_best_valid_acc:
                global_best_valid_acc = valid_acc
                print(f'step_num: {global_step_num}, get new best val_acc: {valid_acc:.5f}, save the model now!')                
                torch.save(model.state_dict(), os.path.join(cfg.model_output_dir, 'best_step_model.pth'))
        
    acc = accuracy(y_pred_lst, y_true_lst)
    return acc

print('ok')

In [ ]:
global_best_train_acc, global_best_valid_acc = 0.0, 0.0
global_train_acc = 0.0
global_step_num = 0

epochs = 10
# optimizer=torch.optim.Adagrad(model.parameters(), lr=0.06)
# optimizer=torch.optim.Adadelta(model.parameters(), lr=10.0)
optimizer=torch.optim.Adam(model.parameters(), lr=0.007, weight_decay=1e-5)
# optimizer=torch.optim.AdamW(model.parameters(), lr=0.007, weight_decay=0.01)
loss_func = nn.BCELoss()

model_ema = None
fgm = None
if cfg.use_ema:
    model_ema = model_EMA(model, decay=0.999)
    model_ema.register()
    
if cfg.use_adversial_training:
    fgm = FGM(model, 'embedding', epsilon=1.0, adv_random=True)
    
for epoch in range(epochs):
    train_acc = train(model, dl_train, optimizer, loss_func, device)
    valid_acc = evaluate(model, dl_valid, device)
    test_acc = evaluate(model, dl_test, device)
    print(f'in epoch: {epoch}, train_acc: {train_acc:.5f}, valid_acc: {valid_acc:.5f}, test_acc: {test_acc:.5f}')
    if train_acc > global_best_train_acc:
        global_best_train_acc = train_acc
    if valid_acc > global_best_valid_acc:
        global_best_valid_acc = valid_acc
        global_train_acc = train_acc
        print(f'at the end of epoch, global_step_num: {global_step_num} get new best_valid_acc: {valid_acc:.5f}, save the model now!')
        torch.save(model.state_dict(), os.path.join(cfg.model_output_dir, 'best_step_model.pth'))

In [ ]:
model = CNN_Net()
model.to(device)

model.load_state_dict(torch.load(os.path.join(cfg.model_output_dir, 'best_step_model.pth')))
test_acc = evaluate(model, dl_test, device)
print(f'final test_acc: {test_acc:.5f}, best_val_acc: {global_best_valid_acc:.5f}, '
      f'train_acc: {global_train_acc:.5f}, best_train_acc: {global_best_train_acc:.5f}')

print('total finished, cost time: ', time.time() - global_start_t)

# no mid_eval  batch_size=20   lr=0.007  Adam  weight_decay=0.0   seed=2033

##################################################################################################

# no mid_eval  batch_size=20   lr=0.007  Adam  weight_decay=0.0   seed=2032
# final test_acc: 0.81667, best_val_acc: 0.82033, train_acc: 0.99050, best_train_acc: 0.99267
# total finished, cost time:  96.26502442359924

# no mid_eval  batch_size=20   lr=0.007  Adam  weight_decay=0.01  seed=2032  不收敛！
# final test_acc: 0.49800, best_val_acc: 0.50400, train_acc: 0.49083, best_train_acc: 0.50783
# total finished, cost time:  95.44412612915039

# no mid_eval  batch_size=20   lr=0.007  Adam  weight_decay=1e-3  seed=2032
# final test_acc: 0.84100, best_val_acc: 0.85033, train_acc: 0.89633, best_train_acc: 0.91183
# total finished, cost time:  96.47475743293762

# no mid_eval  batch_size=20   lr=0.007  Adam  weight_decay=1e-4  seed=2032
# final test_acc: 0.82833, best_val_acc: 0.82433, train_acc: 0.91117, best_train_acc: 0.95617
# total finished, cost time:  135.2124011516571
    
# no mid_eval  batch_size=20   lr=0.007  Adam  weight_decay=1e-5  seed=2032
# final test_acc: 0.82567, best_val_acc: 0.83100, train_acc: 0.88600, best_train_acc: 0.98067
# total finished, cost time:  136.57839918136597

In [ ]:
# with mid_eval  batch_size=4  lr=0.007
# final test_acc: 0.49967, best_val_acc: 0.50733, train_acc: 0.50367, best_train_acc: 0.51283
# total finished, cost time:  188.66995453834534
    
# with mid_eval  batch_size=4  lr=0.006
# final test_acc: 0.51067, best_val_acc: 0.51967, train_acc: 0.49783, best_train_acc: 0.50883
# total finished, cost time:  264.76468110084534

# with mid_eval  batch_size=4  lr=0.001
# final test_acc: 0.81800, best_val_acc: 0.82767, train_acc: 0.99233, best_train_acc: 0.99467
# total finished, cost time:  265.68268299102783
    
# with mid_eval  batch_size=8  lr=0.006
# final test_acc: 0.54067, best_val_acc: 0.53467, train_acc: 0.00000, best_train_acc: 0.52800
# total finished, cost time:  433.7452108860016

# with mid_eval  batch_size=8  lr=0.005
# final test_acc: 0.52200, best_val_acc: 0.52833, train_acc: 0.00000, best_train_acc: 0.63483
# total finished, cost time:  431.8099343776703

# with mid_eval  batch_size=8  lr=0.004
# final test_acc: 0.82100, best_val_acc: 0.82467, train_acc: 0.00000, best_train_acc: 0.99100
# total finished, cost time:  440.1590278148651
    
# with mid_eval  batch_size=8  lr=0.003
# final test_acc: 0.82333, best_val_acc: 0.83333, train_acc: 0.00000, best_train_acc: 0.99450
# total finished, cost time:  537.3426532745361
    
# with mid_eval  batch_size=8  lr=0.002
# final test_acc: 0.82400, best_val_acc: 0.83167, train_acc: 0.00000, best_train_acc: 0.99333
# total finished, cost time:  433.7970836162567

# with mid_eval  batch_size=8  lr=0.001
# final test_acc: 0.81400, best_val_acc: 0.82200, train_acc: 0.00000, best_train_acc: 1.00000
# total finished, cost time:  434.2089605331421
    
# with mid_eval  batch_size=20   lr=0.005
# final test_acc: 0.82900, best_val_acc: 0.82900, train_acc: 0.00000, best_train_acc: 1.00000
# total finished, cost time:  194.9231276512146

# with mid_eval  batch_size=20   lr=0.007  AdamW  weight_decay=0.01
# final test_acc: 0.82500, best_val_acc: 0.82467, train_acc: 0.97900, best_train_acc: 0.98717
# total finished, cost time:  97.36473631858826


    
# with mid_eval  batch_size=32   lr=0.005
# final test_acc: 0.81900, best_val_acc: 0.84133, train_acc: 0.57733, best_train_acc: 0.99217
# total finished, cost time:  143.55632901191711

In [ ]:
# Adadelta  lr=1.0  seed=2036  no mid_eval
# final test_acc: 0.72440, best_val_acc: 0.72220, train_acc: 0.89160, best_train_acc: 0.90800
# total finished, cost time:  192.1149570941925

# Adadelta  lr=1.8  seed=2036  no mid_eval
# final test_acc: 0.77940, best_val_acc: 0.78580, train_acc: 0.83680, best_train_acc: 0.89567
# total finished, cost time:  191.68706226348877
    
# Adadelta  lr=2.0  seed=2036  no mid_eval
# final test_acc: 0.79520, best_val_acc: 0.79220, train_acc: 0.88280, best_train_acc: 0.89840
# total finished, cost time:  265.7658565044403

# Adadelta  lr=2.5  seed=2036  no mid_eval
# final test_acc: 0.77540, best_val_acc: 0.78520, train_acc: 0.82920, best_train_acc: 0.87640
# total finished, cost time:  191.7624773979187
    
# Adadelta  lr=3.0  seed=2036  no mid_eval
# final test_acc: 0.78020, best_val_acc: 0.78220, train_acc: 0.84933, best_train_acc: 0.86280
# total finished, cost time:  192.8262016773224

# Adadelta  lr=4.0  seed=2036  no mid_eval
# final test_acc: 0.74100, best_val_acc: 0.75020, train_acc: 0.80387, best_train_acc: 0.80387
# total finished, cost time:  192.1702754497528

# Adadelta  lr=5.0  seed=2036  no mid_eval
# final test_acc: 0.70080, best_val_acc: 0.70720, train_acc: 0.69053, best_train_acc: 0.69053
# total finished, cost time:  192.71857452392578

# Adadelta  lr=10.0  seed=2036  no mid_eval
# final test_acc: 0.50840, best_val_acc: 0.50420, train_acc: 0.50393, best_train_acc: 0.50700
# total finished, cost time:  249.54866433143616

In [ ]:
# Adagrad  lr=0.03  seed=2036  no mid_eval
# final test_acc: 0.80940, best_val_acc: 0.81160, train_acc: 0.92600, best_train_acc: 0.93807
# total finished, cost time:  276.79689049720764

# Adagrad  lr=0.04  seed=2036  no mid_eval
# final test_acc: 0.81240, best_val_acc: 0.82040, train_acc: 0.90707, best_train_acc: 0.95013
# total finished, cost time:  261.7913763523102

# Adagrad  lr=0.05  seed=2036  no mid_eval
# final test_acc: 0.82920, best_val_acc: 0.83240, train_acc: 0.92020, best_train_acc: 0.96613
# total finished, cost time:  185.5982859134674

# in epoch: 0, train_acc: 0.53927, valid_acc: 0.64200, test_acc: 0.64580
# get new best_valid_acc: 0.64200, save the model now!
# in epoch: 1, train_acc: 0.73927, valid_acc: 0.78620, test_acc: 0.78740
# get new best_valid_acc: 0.78620, save the model now!
# in epoch: 2, train_acc: 0.82847, valid_acc: 0.81080, test_acc: 0.80860
# get new best_valid_acc: 0.81080, save the model now!
# in epoch: 3, train_acc: 0.86187, valid_acc: 0.82840, test_acc: 0.82320
# get new best_valid_acc: 0.82840, save the model now!
# in epoch: 4, train_acc: 0.88567, valid_acc: 0.81700, test_acc: 0.81460
# in epoch: 5, train_acc: 0.90407, valid_acc: 0.82420, test_acc: 0.82340
# in epoch: 6, train_acc: 0.92020, valid_acc: 0.83240, test_acc: 0.82920
# get new best_valid_acc: 0.83240, save the model now!
# in epoch: 7, train_acc: 0.93720, valid_acc: 0.83040, test_acc: 0.82900
# in epoch: 8, train_acc: 0.95140, valid_acc: 0.83140, test_acc: 0.82880
# in epoch: 9, train_acc: 0.96613, valid_acc: 0.82900, test_acc: 0.82960

# Adagrad  lr=0.06  seed=2036  no mid_eval
# final test_acc: 0.50240, best_val_acc: 0.50160, train_acc: 0.50373, best_train_acc: 0.50387
# total finished, cost time:  262.0888602733612

In [ ]:
# Adam  lr=0.005  seed=2036  no mid_eval
# final test_acc: 0.84520, best_val_acc: 0.85200, train_acc: 0.93880, best_train_acc: 0.99280
# total finished, cost time:  189.37609314918518

# AdamW  lr=0.005  seed=2036  no mid_eval
# final test_acc: 0.84800, best_val_acc: 0.85580, train_acc: 0.92647, best_train_acc: 0.99747
# total finished, cost time:  193.3394296169281

In [ ]:
# Adam  lr=0.005  seed=2035  no mid_eval
# final test_acc: 0.84640, best_val_acc: 0.85240, train_acc: 0.92160, best_train_acc: 0.99400
# total finished, cost time:  199.5630133152008

# AdamW  lr=0.005  seed=2035  mid_eval
# final test_acc: 0.85720, best_val_acc: 0.85860, train_acc: 0.90340, best_train_acc: 0.99813
# total finished, cost time:  270.2926092147827

In [ ]:
# Adagrad lr=0.02  seed=2034
# final test_acc: 0.77740, best_val_acc: 0.76220, train_acc: 0.93720, best_train_acc: 0.94513
# total finished, cost time:  282.7268285751343

# AdamW  lr=0.005  seed=2034
# final test_acc: 0.85700, best_val_acc: 0.85380, train_acc: 0.90427, best_train_acc: 1.00000
# total finished, cost time:  292.4551508426666

# Adam  lr=0.001  seed=2034
# final test_acc: 0.80540, best_val_acc: 0.79580, train_acc: 0.96187, best_train_acc: 0.99933
# total finished, cost time:  289.06110072135925

# Adam  lr=0.003  seed=2034
# final test_acc: 0.83780, best_val_acc: 0.84600, train_acc: 0.93907, best_train_acc: 1.00000
# total finished, cost time:  285.3573806285858

# Adam  lr=0.005  seed=2034
# final test_acc: 0.85160, best_val_acc: 0.85500, train_acc: 0.89727, best_train_acc: 0.99993
# total finished, cost time:  284.3437497615814

# Adam  lr=0.007  seed=2034
# final test_acc: 0.83020, best_val_acc: 0.83620, train_acc: 0.87953, best_train_acc: 0.98880
# total finished, cost time:  283.920378446579

# Adam  lr=0.01  seed=2034
# final test_acc: 0.50240, best_val_acc: 0.50160, train_acc: 0.50433, best_train_acc: 0.50433
# total finished, cost time:  267.2974200248718